# Построение филогенетических деревьев на Python 
Для построения деревьев мы будем использовать библиотеку ete3.

In [1]:
# Установка
!pip install ete3

## Основы ete3

In [2]:
from ete3 import Tree

In [3]:
# Загружает структуру дерева из строки. Возвращает переменную ’t’ - корневой узел дерева.
t = Tree("(A:1,(B:1,(E:1,D:1):0.5):0.5);" )

# Загружает структуру дерева из newick файла.
t = Tree("genes_tree.nh")

# Вы также можете указать формат newick. Например, для именованных внутренних узлов мы будем использовать format 1.
t = Tree("(A:1,(B:1,(E:1,D:1)Internal_1:0.5)Internal_2:0.5)Root;", format=1)

In [4]:
# Печатает newick используя стандартный формат
print(t.write())

# Чтобы печатать имена внутренних узлов, вам нужно поменять формат (format=1):
print(t.write(format=1))

# Также можно записывать в файл
t.write(format=1, outfile="new_tree.nw")

(A:1,(B:1,(E:1,D:1)1:0.5)1:0.5);
(A:1,(B:1,(E:1,D:1)Internal_1:0.5)Internal_2:0.5);


In [5]:
# Лист, состоящий из детей (тип - ete3.coretype.tree.TreeNode)
t.children

# Лист, состоящий из листов (узлов без детей, тип - ete3.coretype.tree.TreeNode)
t.get_leaves()

[Tree node 'A' (-0x7fffffe1ae341f58),
 Tree node 'B' (-0x7fffffe1ae341fdc),
 Tree node 'E' (-0x7fffffe1ae341f54),
 Tree node 'D' (-0x7fffffe1ae336bb8)]

In [6]:
t = Tree("(A:1,(B:1,(E:1,D:1)Internal_1:0.5)Internal_2:0.5)Root;", format=1)
print(t)

t.children[1].delete()
print(t)


   /-A
--|
  |   /-B
   \-|
     |   /-E
      \-|
         \-D

   /-A
  |
--|--B
  |
  |   /-E
   \-|
      \-D


In [7]:
t = Tree("(A:1,(B:1,(E:1,D:1)Internal_1:0.5)Internal_2:0.5)Root;", format=1)
print(t)

t.children[1].add_sister(name="K")
print(t)


node = t.children[1].children[0]

# Добавление ребенка к текущему узлу
node.add_child(name="M")
node.add_child(name="N")
print(t)


   /-A
--|
  |   /-B
   \-|
     |   /-E
      \-|
         \-D

   /-A
  |
  |   /-B
  |--|
--|  |   /-E
  |   \-|
  |      \-D
  |
   \-K

   /-A
  |
  |      /-M
  |   /-|
  |  |   \-N
--|--|
  |  |   /-E
  |   \-|
  |      \-D
  |
   \-K


In [8]:
# Полный список методов 
#help(t)

## Построение филогенетического дерева

In [9]:
from ete3 import PhyloTree
fasta_txt = """
>seqA
MAEIPDETIQQFMALT---HNIAVQYLSEFGDLNEALNSYYASQTDDIKDRREEAH
>seqB
MAEIPDATIQQFMALTNVSHNIAVQY--EFGDLNEALNSYYAYQTDDQKDRREEAH
>seqC
MAEIPDATIQ---ALTNVSHNIAVQYLSEFGDLNEALNSYYASQTDDQPDRREEAH
>seqD
MAEAPDETIQQFMALTNVSHNIAVQYLSEFGDLNEAL--------------REEAH
"""

# Загружаем дерево и связыеваем листья с последовательностями
t = PhyloTree("(((seqA,seqB),seqC),seqD);", alignment=fasta_txt, alg_format="fasta")

print(t)


         /-seqA
      /-|
   /-|   \-seqB
  |  |
--|   \-seqC
  |
   \-seqD


## Визуализация филогенетического дерева

In [10]:
from ete3 import PhyloTree, TreeStyle

alg = """
 >Dme_001
 MAEIPDETIQQFMALT---HNIAVQYLSEFGDLNEAL--YYASQTDDIKDRREEAH
 >Dme_002
 MAEIPDATIQQFMALTNVSHNIAVQY--EFGDLNEALNSYYAYQTDDQKDRREEAH
 >Cfa_001
 MAEIPDATIQ---ALTNVSHNIAVQYLSEFGDLNEALNSYYASQTDDQPDRREEAH
 >Mms_001
 MAEAPDETIQQFMALTNVSHNIAVQYLSEFGDLNEAL--------------REEAH
 >Hsa_001
 MAEIPDETIQQFMALT---HNIAVQYLSEFGDLNEALNSYYASQTDDIKDRREEAH
 >Ptr_002
 MAEIPDATIQ-FMALTNVSHNIAVQY--EFGDLNEALNSY--YQTDDQKDRREEAH
 >Mmu_002
 MAEIPDATIQ---ALTNVSHNIAVQYLSEFGDLNEALNSYYASQTDDQPDRREEAH
 >Hsa_002
 MAEAPDETIQQFM-LTNVSHNIAVQYLSEFGDLNEAL--------------REEAH
 >Mmu_001
 MAEIPDETIQQFMALT---HNIAVQYLSEFGDLNEALNSYYASQTDDIKDRREEAH
 >Ptr_001
 MAEIPDATIQ-FMALTNVSHNIAVQY--EFGDLNEALNSY--YQTDDQKDRREEAH
"""

def get_example_tree():

    # Выполняет анализ согласования дерева
    gene_tree_nw = '((Dme_001,Dme_002),(((Cfa_001,Mms_001),((Hsa_001,Ptr_001),Mmu_001)),(Ptr_002,(Hsa_002,Mmu_002))));'
    species_tree_nw = "((((Hsa, Ptr), Mmu), (Mms, Cfa)), Dme);"
    genetree = PhyloTree(gene_tree_nw)
    sptree = PhyloTree(species_tree_nw)
    recon_tree, events = genetree.reconcile(sptree)
    recon_tree.link_to_alignment(alg)
    return recon_tree, TreeStyle()

if __name__ == "__main__":
    # Визуализируйте согласованное дерево
    t, ts = get_example_tree()
    t.show(tree_style=ts)
    t.render("phylotree.png", w=750)

Warnning: [5] terminal nodes could not be found in the alignment.
